In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras import backend
import sys
import h5py

In [ ]:
# To use GPU, run this to check GPU is set up
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
sys.path.insert(0,'/content/drive/My Drive/CS733-HW4/')
root_path=sys.path[0]
root_path

'/content/drive/My Drive/CS733-HW4/'

In [ ]:
def load_dataset():
    train_dataset = h5py.File(f'{root_path}datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    #train_set_y_orig is a 1D array (with one row, just like a list), which is different from a 2D array with only one row

    test_dataset = h5py.File(f'{root_path}datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    #Transforming y arrays (1D) to 2D arrays (with only one row)
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
# Load and preprocess data
X_train, y_train, X_test, y_test, classes = load_dataset()

y_train = y_train.T
y_test = y_test.T

In [ ]:
# print("Original X shape: ", X_train.shape)
# print("X data type:", X_train.dtype)
# print("Y shape: ", y_train.shape)

In [15]:
# Construct a CNN network
model = Sequential(
    [
      Conv2D(filters=32, kernel_size=3, padding= 'same', activation='relu', input_shape=(64,64,3)),
      BatchNormalization(),
      MaxPooling2D(),
      Conv2D(filters=64, kernel_size=3, padding= 'same', activation='relu'),
      BatchNormalization(),
      MaxPooling2D(),
      Conv2D(filters=128, kernel_size=3, padding= 'same', activation='relu'),
      BatchNormalization(),
      MaxPooling2D(),
      Conv2D(filters=128, kernel_size=3, padding= 'same', activation='relu'),
      BatchNormalization(),
      MaxPooling2D(),
      Flatten(),
      Dropout(0.5),
      Dense(512, activation="relu"),
      Dense(1, activation="sigmoid")
    ]
)

# Get the summary of each layer
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 128)      

In [ ]:
# Compile model
# custom_opt = keras.optimizers.Adagrad(learning_rate=0.0001)
# model.compile(optimizer=custom_opt, loss="binary_crossentropy", metrics=["accuracy"])

model.compile(optimizer='nadam', loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# checkpointer = keras.callbacks.ModelCheckpoint("cats_model.h5", save_best_only=True)
# earlystopper = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

# history = model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[checkpointer, earlystopper], verbose=0)

In [ ]:
# Train model for 100 epochs
history = model.fit(X_train, y_train, epochs=100, validation_split=0.1, verbose=0)

In [ ]:
print("Training accuracy:")
model.evaluate(X_train, y_train)
print("Testing accuracy:")
model.evaluate(X_test, y_test)

Training accuracy:
7/7 [==============================] - 0s 8ms/step - loss: 0.1978 - accuracy: 0.9761
Testing accuracy:
2/2 [==============================] - 0s 32ms/step - loss: 0.6936 - accuracy: 0.9400


[0.6935617327690125, 0.9399999976158142]

In [ ]:
# model.save_weights(f'{root_path}/cnn_cats_model_best_weights.h5')
# model.save(f'{root_path}/cnn_cats_model_best.h5')

## Best model
Training accuracy: 97.61
Testing accuracy: 94.00